# Implementación Count-Min Sketch
Natalia Ruiz Martínez

In [ ]:
import math
import numpy as np
import random

class CountMinSketch:

  # Constructor de la clase
  def __init__(self, epsilon, delta):

    # Comprueba si los valores de epsilon y delta se encuentran dentro del intervalo correcto
    if epsilon <= 0 or epsilon >= 1:
        raise ValueError("Se debe establece epsilon entre 0 y 1")
    if delta <= 0 or delta >= 1:
        raise ValueError("Se debe establece delta entre 0 y 1")

    # Definición del número de filas (d) y columnas (w)
    self.d = math.ceil(np.log(1/delta))
    self.w = math.ceil(math.e/epsilon)

    # Se crea una familia de d funciones a partir del método hash_function, que por su construcción garantiza que la familia sea 2-independiente
    self.hash_family = [self.hash_function() for _ in range(self.d)]

    # Se crea la matriz sketch C y se inicializa en cero
    self.C = np.zeros((self.d, self.w), int)


  # Se actualiza la matriz contando todos los elementos del dataset
  def update(self, dataset):

    for element in dataset:

      for j in range(self.d):

        # Para poder aplicar las funciones creadas con hash_function, el elemento debe estar en formato numérico,
        # luego se aplica hash (que devuelve un hash del elemento positivo o negativo) y se le aplica el valor absoluto
        self.C[j, self.hash_family[j](abs(hash(element)))] += 1


  # Se calcula la frecuencia de element en el dataset
  def frecuency(self, element):

    # Se guarda el valor de cada fila de la matriz correspondiente a la aplicación de la función hash sobre el elemento
    min_count = [self.C[j, self.hash_family[j](abs(hash(element)))] for j in range(self.d)]

    # Se devuelve el mínimo de todos los valores guardados, que corresponde con la estimación de la frecuencia
    return min(min_count)


  # Este método devuelve una función
  def hash_function(self):

    # Se elige un número primo grande aleatorio p, por ejemplo, el siguiente
    p = 2**61 - 1

    # Se eligen dos enteros a y b de forma aleatoria entre 0 y el número primo p
    a = random.randint(0, p)
    b = random.randint(0, p)

    # Se devuelve una función que dado un elemento (en forma numérica) devuelve un entero entre 0 y w-1
    return lambda element: (a * element + b) % p % self.w

Importamos el dataset Airline Passenger Satisfaction de Kaggle.

In [ ]:
from google.colab import files
files.upload() # importar kaggle.json

In [ ]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d teejmahal20/airline-passenger-satisfaction
!unzip airline-passenger-satisfaction.zip

  0% 0.00/2.71M [00:00<?, ?B/s]
100% 2.71M/2.71M [00:00<00:00, 231MB/s]
Archive:  airline-passenger-satisfaction.zip
  inflating: test.csv                
  inflating: train.csv               


Los datos están divididos en dos grupos, en este caso usaremos train.csv para comprobar que el algoritmo anterior funciona correctamente. Para ello, comprobamos previamente cuantos datos de cada tipo distinto existen en Class.

In [ ]:
import pandas as pd
data = pd.read_csv('train.csv')
data['Class'].value_counts()

Business    49665
Eco         46745
Eco Plus     7494
Name: Class, dtype: int64

A continuación, creamos un objeto de clase CountMinSketch, establecemos $\epsilon = 0.005$ y $\delta = 10^{-7}$, definimos el dataset, actualizamos la matriz con los datos y obtenemos la frecuencia del elemento Business.

In [ ]:
sketch = CountMinSketch(epsilon=0.005, delta=10**-7)

In [ ]:
dataset = data['Class']

In [ ]:
sketch.update(dataset)

In [ ]:
sketch.frecuency('Business')

49665

Luego el algoritmo ha estimado correctamente la frecuencia del elemento.